In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class ImageEditor:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Resize & Drag-to-Crop Tool")

        self.image = None
        self.tk_img = None
        self.display_image = None

        self.label = tk.Label(root, text="No image selected")
        self.label.pack()

        self.canvas = tk.Canvas(root, width=400, height=400, cursor="cross")
        self.canvas.pack()

        tk.Button(root, text="Open Image", command=self.open_image).pack(pady=5)

        # Resize Inputs
        tk.Label(root, text="Resize (width x height):").pack()
        self.resize_width = tk.Entry(root, width=10)
        self.resize_width.pack(side="left", padx=5)
        self.resize_height = tk.Entry(root, width=10)
        self.resize_height.pack(side="left", padx=5)
        tk.Button(root, text="Resize", command=self.resize_image).pack(pady=5)

        tk.Button(root, text="Save Image", command=self.save_image).pack(pady=5)

        # Crop by drag
        self.start_x = self.start_y = 0
        self.rect_id = None
        self.canvas.bind("<ButtonPress-1>", self.start_crop)
        self.canvas.bind("<B1-Motion>", self.update_crop)
        self.canvas.bind("<ButtonRelease-1>", self.perform_crop)

    def open_image(self):
        file_path = filedialog.askopenfilename()
        if not file_path:
            return
        self.image = Image.open(file_path)
        self.original_image = self.image.copy()
        self.show_image()
        self.label.config(text=file_path)

    def show_image(self):
        resized = self.image.resize((400, 400))
        self.tk_img = ImageTk.PhotoImage(resized)
        self.canvas.create_image(0, 0, anchor="nw", image=self.tk_img)

    def resize_image(self):
        if self.image:
            try:
                w = int(self.resize_width.get())
                h = int(self.resize_height.get())
                self.image = self.image.resize((w, h))
                self.original_image = self.image.copy()
                self.show_image()
            except ValueError:
                messagebox.showerror("Invalid input", "Width and height must be integers.")

    def start_crop(self, event):
        self.start_x = event.x
        self.start_y = event.y
        if self.rect_id:
            self.canvas.delete(self.rect_id)
        self.rect_id = self.canvas.create_rectangle(self.start_x, self.start_y, self.start_x, self.start_y, outline='red')

    def update_crop(self, event):
        self.canvas.coords(self.rect_id, self.start_x, self.start_y, event.x, event.y)

    def perform_crop(self, event):
        end_x, end_y = event.x, event.y
        # Convert canvas coordinates back to original image size
        x_scale = self.original_image.width / 400
        y_scale = self.original_image.height / 400

        x1 = int(min(self.start_x, end_x) * x_scale)
        y1 = int(min(self.start_y, end_y) * y_scale)
        x2 = int(max(self.start_x, end_x) * x_scale)
        y2 = int(max(self.start_y, end_y) * y_scale)

        if self.image:
            self.image = self.original_image.crop((x1, y1, x2, y2))
            self.original_image = self.image.copy()
            self.show_image()

    def save_image(self):
        if self.image:
            file_path = filedialog.asksaveasfilename(defaultextension=".jpg",
                                                     filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png")])
            if file_path:
                self.image.save(file_path)
                messagebox.showinfo("Saved", f"Image saved to:\n{file_path}")

# Run the app
root = tk.Tk()
app = ImageEditor(root)
root.mainloop()
